In [1]:
import numpy as np
from scipy.optimize import least_squares

# -------------------------------
# 1) Beispiel-Daten
# -------------------------------
# Wellenlängen in µm (z.B. 3 Kanäle)
lam_um = np.array([8.0, 10.0, 12.0])

# Umgebungsobjekte: 2 Materialien × 3 Wellenlängen (Radiance-Einheiten konsistent!)
S_EnvObj = np.array([
    [1.20, 1.00, 0.80],  # Himmel
    [0.60, 0.70, 0.90],  # Vegetation
])

# Sichtanteile (Summe = 1): 70% Himmel, 30% Vegetation
# Wie stark jedes Objkekt zu sehen ist.
V = np.array([0.7, 0.3])

# Emissivität des Zielobjekts (pro Wellenlänge)
eps = np.array([0.95, 0.90, 0.85]) # Emissivität
rho = 1.0 - eps # Reflexion anteil 

# -------------------------------
# 2) Umgebungsstrahlung X(λ)
# 3) gemischte Umgebungsstrahlung der gesammte Umgebung
# 4) Wird gewichtet nach dem Einfluss jedes Objektes und der Emissivität des Zielobjektes
# -------------------------------
X = (V @ S_EnvObj).T  # -> (3,) = gemischte Umgebungsstrahlung der gesammte Umgebung
# Alternativ explizit: X = (V @ S_EnvObj)  (hier bereits 1D mit Länge 3)
# array([1.02, 0.91, 0.83]) Hier sind Himmel und Vegetation zusammen gemischt

# -------------------------------
# 3) Planck-Funktion B(λ, T)
#    (λ in µm; Ergebnis in W·m^-3·sr^-1; für viele Anwendungen reicht relative Form)
# -------------------------------
def planck_lambda(lam_um, T):
    h  = 6.62607015e-34
    c  = 2.99792458e8
    kB = 1.380649e-23
    lam = lam_um * 1e-6  # µm -> m
    num = 2*h*c**2 / lam**5
    den = np.exp((h*c)/(lam*kB*T)) - 1.0
    return num / den  # Achtung: Einheiten ggf. auf W/(m^2 sr µm) skalieren, falls nötig.

# -------------------------------
# 4) Vorwärtsmodell: S_meas = eps*B + (1-eps)*X
#    -> Wir simulieren Messung mit einer "wahren" Temperatur T_true
# -------------------------------
T_true = 305.0  # K (nur Beispiel)
B_true = planck_lambda(lam_um, T_true)
S_emit_true = eps * B_true  # Das wahre Starhlung von uns oder Objekt die wir identifizieren wollen
S_refl_true = rho * X # Die Reflexion der Umgebung (1-eps)*X
S_meas = S_emit_true + S_refl_true

# Optional: Messrauschen hinzufügen
# rng = np.random.default_rng(42)
# S_meas = S_meas * (1.0 + 0.005*rng.standard_normal(S_meas.shape))

# -------------------------------
# 5) Inversion: T schätzen aus S_meas, eps und X
# -------------------------------
def residual_T(T):
    B = planck_lambda(lam_um, T[0])
    model = eps * B + rho * X
    return S_meas - model  # Least-Squares-Residuum


"""
max_nfev=200

Das bedeutet nur:

Der Solver darf höchstens 200 Funktionsaufrufe machen (also 200 Temperaturversuche).

Er kann vorher aufhören, wenn er schon eine gute Lösung gefunden hat.

"""

res = least_squares(residual_T, x0=[300.0], bounds=(200.0, 500.0), max_nfev=200)
T_est = float(res.x[0])

# -------------------------------
# 6) Ergebnisse ausgeben
# -------------------------------
print("Wellenlängen [µm]:", lam_um)
print("X(λ)  (Umgebungsstrahlung):", np.round(X, 6))
print("rho(λ)=1-eps(λ):", rho)
print("S_meas(λ):", np.round(S_meas, 6))
print(f"T_true = {T_true:.2f} K,  T_est = {T_est:.2f} K")
print(f"Fit-Restfehler (Sum of squares) = {res.cost:.3e}")


Wellenlängen [µm]: [ 8. 10. 12.]
X(λ)  (Umgebungsstrahlung): [1.02 0.91 0.83]
rho(λ)=1-eps(λ): [0.05 0.1  0.15]
S_meas(λ): [9517527.098191 9668782.258248 8143263.694765]
T_true = 305.00 K,  T_est = 305.00 K
Fit-Restfehler (Sum of squares) = 2.307e-16


In [2]:
rho # Reflexion (1- e)

array([0.05, 0.1 , 0.15])

In [6]:
eps # Emissivität

array([0.95, 0.9 , 0.85])

In [3]:
X  # (V @ S_EnvObj).T Gemischte Strahlung

array([1.02, 0.91, 0.83])

In [7]:
S_refl_true # Die Reflexion der Umgebung (1-eps)*X

array([0.051 , 0.091 , 0.1245])

In [4]:
B_true # Gesammte Starhlung in eine Pixel (Beispiel) ohne emissivität abzuziehen

array([10018449.52335848, 10743091.29694198,  9580310.08266512])

In [5]:
S_emit_true # Das wahre Starhlung  eps*B(T) 

array([9517527.04719056, 9668782.16724778, 8143263.57026535])

In [4]:
res.x

array([305.])

In [5]:
import numpy as np
from scipy.optimize import least_squares

# ============================================================
# 0) DEMO-SETUP (dein Beispiel, minimal erweitert)
#    -> Wir bleiben bei 3 Bändern, 2 Umweltobjekten, 1 Pixel
# ============================================================

# Wellenlängen in µm
lam_um = np.array([8.0, 10.0, 12.0])

# Umgebungsobjekte: 2 Materialien × 3 Wellenlängen (Radiance in konsistenten Einheiten)
S_EnvObj = np.array([
    [1.20, 1.00, 0.80],  # Himmel
    [0.60, 0.70, 0.90],  # Vegetation
])

# Sichtanteile (Summe = 1): wie stark jedes Umweltobjekt "gesehen" wird
V = np.array([0.7, 0.3])  # 70% Himmel, 30% Vegetation

# Emissivität des Zielobjekts (bandweise). In echt: aus matLib[:, idx] oder als Mischung.
eps = np.array([0.95, 0.90, 0.85])
rho = 1.0 - eps  # Reflexionsfaktor nach Kirchhoff (opak, tau≈0)

# Gemischte Umgebungsstrahlung X(λ) = V^T S_EnvObj
X = (V @ S_EnvObj).T  # -> (3,) Umgebungsstrahlung pro Band

# -------------------------------
# Planck-Funktion B(λ, T)
# λ in µm; Output in W·m^-3·sr^-1 (Skalierung ggf. anpassen zu deiner Radiometrie)
# -------------------------------
def planck_lambda(lam_um, T):
    h  = 6.62607015e-34
    c  = 2.99792458e8
    kB = 1.380649e-23
    lam = lam_um * 1e-6  # µm -> m
    num = 2*h*c**2 / lam**5
    den = np.exp((h*c)/(lam*kB*T)) - 1.0
    return num / den  # Für relative Fits oft ausreichend

# -------------------------------
# FORWARD: Wir simulieren eine Messung S_meas
# -------------------------------
T_true = 305.0  # K (nur Demo)
B_true = planck_lambda(lam_um, T_true)
S_emit_true = eps * B_true          # Eigenstrahlung
S_refl_true = rho * X               # Reflexion der Umgebung
S_meas = S_emit_true + S_refl_true  # gemessenes Spektrum (modelliert)

# ============================================================
# 1) PHYSIKALISCHE STRUKTUR: T, S_emit, S_refl, eps_est
# ============================================================

# (a) Temperatur-Fit (Least Squares): min || S_meas - (eps*B(T) + rho*X) ||^2
def residual_T(T):
    B = planck_lambda(lam_um, T[0])
    model = eps * B + rho * X
    return S_meas - model

res = least_squares(residual_T, x0=[300.0], bounds=(200.0, 500.0), max_nfev=200)
T_est = float(res.x[0])

# (b) Mit T_est Komponenten trennen
B_est = planck_lambda(lam_um, T_est)
S_refl_est = rho * X
S_emit_est = S_meas - S_refl_est  # eigen-emittierter Anteil (modellbasiert)

# (c) Emissivität aus umgestellter Gleichung schätzen (optional Check):
#     eps ≈ (S_meas - X) / (B(T) - X), bandweise, auf [0,1] clippen
eps_est = (S_meas - X) / np.maximum(B_est - X, 1e-12)
eps_est = np.clip(eps_est, 0.0, 1.0)

# ============================================================
# 2) CHEMISCHE STRUKTUR: einfacher Fingerprint-Score Γ
#    Idee: reflektierte Umweltstrahlung trägt chemische "Linienformen".
#    Für die Demo definieren wir eine Referenzsignatur mit Peak bei 10 µm.
#    In echt: nimm Referenzspektren/Peaks (z.B. Sprengstoff, Plastik) und match sie.
# ============================================================

# Referenz-"Resonanz"-Form (Demo): Peak bei λ=10 µm
gamma_ref = np.array([0.2, 1.0, 0.2])  # sehr simpel (nur Demo)

# Wir nehmen als "chemie-tragend" den reflektierten Teil (kannst auch andere Normierungen testen)
# Normalisierung: auf Norm 1 bringen für Cosine Similarity
def normalize(v, eps=1e-12):
    n = np.linalg.norm(v)
    return v / (n + eps)

chem_vec = normalize(S_refl_est)
chem_ref = normalize(gamma_ref)

# Cosine Similarity als Score ([-1..1], hier ~[0..1] wegen Positivität)
Gamma_score = float(np.dot(chem_vec, chem_ref))

# ============================================================
# 3) TIEFENSTRUKTUR: Schätzung z via Beer–Lambert (Demo)
#    Annahme: eine Anomalie A(λ) ~ K * exp(-α(λ) * z)
#    Dann: log(A) = log(K) - α(λ) * z  -> linearer Fit in α
#    Wir bauen hier eine simple Anomalie aus reflektiertem Anteil (Demo!),
#    in echt: nimm z.B. die "minen-spezifische" Komponente (dein TLDE-ΔB* * Γ).
# ============================================================

# Demo-Absorptionskoeffizienten α(λ) (frei gewählt; in echt: material- & erdabhängig)
alpha = np.array([0.8, 1.2, 1.6])  # 1/m (nur Demo)

# "Anomalie" A(λ): nimm einen strictly-positiven Teil, z.B. reflektierte Komponente
A = np.maximum(S_refl_est, 1e-9)
y = np.log(A)            # log(A)
x = alpha                # α

# Linearer Fit y = c - z * α  -> slope = -z
# Bei 3 Punkten: einfache lineare Regression
Xmat = np.vstack([np.ones_like(x), -x]).T   # [1, -α] @ [c, z]^T
coef, *_ = np.linalg.lstsq(Xmat, y, rcond=None)
c_fit, z_est = coef[0], coef[1]             # z_est >=0 erwartet, wenn A mit α fällt

# ============================================================
# 4) GEOMETRISCHE STRUKTUR (Ausblick)
#    Für EIN Pixel nicht machbar. Auf Bild/Map-Ebene:
#    - Segmentierung -> Masken
#    - Blob-/Konturanalyse -> runde Flecken? Größe? Ränder?
#    - Polarisation (optional) -> glanz/gerichtete Reflexion trennen
#    Unten Pseudocode.
# ============================================================
"""
# PSEUDOCODE (auf Bildebene):
seg_map = segment_image(rgb_or_hsi)            # Semantic Segmentation
V_map    = area_fractions_per_region(seg_map)  # pro Patch/Region: V bestimmen
X_map    = mix_env_stradiance(V_map, S_EnvObj) # Umgebungsspektrum pro Region
eps_map  = lookup_eps_from_matLib(labels)      # oder Mischung/Schätzung
T_map    = fit_temperature_per_pixel(S_meas_cube, eps_map, X_map)

# Geometrie:
anomaly = detect_anomaly(T_map or Γ_map)       # z.B. DoG/LoG/Threshold/Connected Components
features = measure_shape(anomaly)              # Fläche, Rundheit, Exzentrizität, Kanten-Schärfe
"""

# ============================================================
# 5) AUSGABE
# ============================================================
print("=== Physikalische Struktur ===")
print("Wellenlängen [µm]:        ", lam_um)
print("X(λ) Umgebung:            ", np.round(X, 6))
print("rho(λ)=1-eps(λ):          ", np.round(rho, 6))
print("S_meas(λ):                ", np.round(S_meas, 6))
print(f"T_true = {T_true:.2f} K,   T_est = {T_est:.2f} K")
print("S_emit_est(λ) (Eigen):    ", np.round(S_emit_est, 6))
print("S_refl_est(λ) (Reflex):   ", np.round(S_refl_est, 6))
print("eps_est (umgestellt):     ", np.round(eps_est, 6))
print(f"Fit-Restfehler (SSE)     = {res.cost:.3e}")

print("\n=== Chemische Struktur (Demo) ===")
print("gamma_ref (Peak@10µm):    ", gamma_ref)
print(f"Gamma-Score (cosine)     = {Gamma_score:.3f}  (≈0..1; höher = ähnlicher)")

print("\n=== Tiefenstruktur (Demo) ===")
print("alpha(λ) [1/m]:           ", alpha)
print("A(λ) (Anomalie, Demo):    ", np.round(A, 6))
print(f"z_est (aus log-Fit)      = {z_est:.3f} m   (nur Demo-Formel)")


=== Physikalische Struktur ===
Wellenlängen [µm]:         [ 8. 10. 12.]
X(λ) Umgebung:             [1.02 0.91 0.83]
rho(λ)=1-eps(λ):           [0.05 0.1  0.15]
S_meas(λ):                 [9517527.098191 9668782.258248 8143263.694765]
T_true = 305.00 K,   T_est = 305.00 K
S_emit_est(λ) (Eigen):     [9517527.047191 9668782.167248 8143263.570265]
S_refl_est(λ) (Reflex):    [0.051  0.091  0.1245]
eps_est (umgestellt):      [0.95 0.9  0.85]
Fit-Restfehler (SSE)     = 2.307e-16

=== Chemische Struktur (Demo) ===
gamma_ref (Peak@10µm):     [0.2 1.  0.2]
Gamma-Score (cosine)     = 0.747  (≈0..1; höher = ähnlicher)

=== Tiefenstruktur (Demo) ===
alpha(λ) [1/m]:            [0.8 1.2 1.6]
A(λ) (Anomalie, Demo):     [0.051  0.091  0.1245]
z_est (aus log-Fit)      = -1.116 m   (nur Demo-Formel)
